## Introduction

The `typing` module enables you to specify what *types* of inputs and outputs you expect a functions parameters and return values to be.

For example:

```python
def greeting(name: str) -> str
    return "Hello" + name
```

Also enables you to `def`ine your own types from more primitive types

```python
Vector = list[str]
```

As well as define and nest more complex data structures easily and comprehensibily

------------------------------------------

Now `pybullet` is a module used for:
* Forward kinematics
* Inverse kinamatics
* Collision detection

For more information about pybullet functions, see [documentation](https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#) 

Remember the syntax for initializing an object in python (class constructor):

```python
def __init__(self, param1, param2):
    self.param1 = param1
    self.param2 = param2
```

Here we create aliases for two types of data structures. Specifically 2-tuples of floats (e.g. (5.2, 6.1))
* Bounds = Give the *interval of rotation* a given revolute can rotate [e.g. (-pi/2,pi/2)]
* XY = *(x,y)-coordinates* of the end effector
* Relative Pose of each revolute (given by an angle)


In [2]:
from typing import Any, Optional
import pybullet as p

In [3]:
BOUNDS = tuple[float, float]
XY = tuple[float, float]
CONFIG = tuple[float, float]

## Robot2R

In [4]:
import math 
class Robot2R:
    CONFIG = tuple[float, float]

    def __init__(self,
                 link_lengths: tuple[float,
                                     float],
                 bounds: Optional[tuple[BOUNDS,
                                        BOUNDS]] = None) -> None:
                                        self.l1 = link_lengths[0]
                                        self.l2 = link_lengths[1]

    def forward_kinematics(self, configuration: CONFIG) -> XY:
        l1, l2= self.l1, self.l2
        theta1, theta2 = configuration[0], configuration[1]
         
        x = l1*math.cos(theta1) + l2*math.cos(theta1 + theta2)
        y = l1*math.sin(theta1) + l2*math.sin(theta1 + theta2)

        return (x,y)

    def inverse_kinematics(self, end_effector_position: XY) -> CONFIG:
        # a2 and a1 are angles
        # arccosine is always positive
        x, y = end_effector_position[0], end_effector_position[1]
        l1, l2= self.l1, self.l2

        # Be careful about 0 division errors
        a2 = math.acos((x**2 + y**2 - l1**2 - l2**2) / (2*l1*l2))
        if x != 0:
            a1 = math.atan(y/x) - math.atan((l2*math.sin(a2)) / (l1 + l2 * math.cos(a2)))
        else: 
            a1 = math.pi/2 - math.atan((l2*math.sin(a2)) / (l1 + l2 * math.cos(a2)))
        return (a1, a2)
        


Remember the syntax for initializing an object in python (class constructor):

```python
def __init__(self, param1, param2):
    self.param1 = param1
    self.param2 = param2
```

**Testing**

In [5]:
# Length of robot links (rigid bodies)
link_lengths_2R = (50.0, 40.0)

# Creating 2R robot
testRobot = Robot2R(link_lengths_2R)

In [6]:
############ Forward Kinematics Tests #############
pi = math.pi
configuration1 = (pi/2,-pi/2)
test1 = testRobot.forward_kinematics(configuration1)
print(test1, "should be 40, 50")

configuration2 = (0, pi/4)
test2 = testRobot.forward_kinematics(configuration2)
print(test2, "should be", 50 + 40*math.cos(pi/4), 40*math.sin(pi/4))

(40.0, 50.0) should be 40, 50
(78.2842712474619, 28.284271247461902) should be 78.2842712474619 28.284271247461902


In [7]:
############# Inverse Kinematics Tests #############
XY = (43.6, 54.8)
test1 = testRobot.inverse_kinematics(XY)
print(testRobot.forward_kinematics(test1), "should be ", XY)

XY = (0,90)
test2 = testRobot.inverse_kinematics(XY)
print(testRobot.forward_kinematics(test2), "should be ", XY)

(43.6, 54.8) should be  (43.6, 54.8)
(5.5109105961630896e-15, 90.0) should be  (0, 90)


## Robot3R

Here we implement the robot with three revolute joints.

*Specifications*

Link Lengths:  $(60 cm, 40 cm, 30 cm)$

Maximum Angles:  $([-2pi, 2pi], [-3/4pi, 3/4pi], [-3/4pi, 3/4pi])$

In [46]:
import numpy as np

class Robot3R:
    CONFIG = tuple[float, float, float]
    XYZ = tuple[float, float, float]

    def __init__(self,
                 link_lengths: tuple[float, float, float],
                 bounds: Optional[tuple[BOUNDS, BOUNDS, BOUNDS]] = None) -> None:
                 self.l1  = link_lengths[0]
                 self.l2 = link_lengths[1]
                 self.l3 = link_lengths[2]
        

    def forward_kinematics(self, configuration: CONFIG) -> XYZ:
        # Get x,y
        l1, l2,l3  = self.l1, self.l2, self.l3
        a1, a2, a3 = configuration[0], configuration[1], configuration[2]
        xyz = np.zeros(3)
        first_one = np.array([l1*math.cos(a1), l1*math.sin(a1), 0])
        xyz += first_one
        
        rotation_matrix = np.array([
            [math.cos(a1), -1*math.sin(a1), 0],
            [math.sin(a1), math.cos(a1), 0],
            [0,0,1]
        ])

        second_one = np.array([l2*math.cos(a2),0,l2*math.sin(a2)])
        second_one = rotation_matrix.dot(second_one)

        thrid_one = np.array([l3*math.cos(a2 + a3), 0, l3 *math.cos(a2+a3)])
        third_one = rotation_matrix.dot(third_one)

        xyz = second_one + third_one

        return (xyz[0], xyz[1], xyz[2])


    def inverse_kinematics(self, end_effector_position: XYZ) -> CONFIG:
        # (1) Unpacking parameters
        x,y,z = end_effector_position[0], end_effector_position[1], end_effector_position[2]
        l1 = self.l1

        # (2) Find angle of first robot manipulator
        a1 = math.atan(y/x)

        # (3) Rotate problem into two dimensions 
        ## Defining inverse of a1-rotation matrix - (Do not need to define inverse because)
        rotation_matrix = np.array([
            [math.cos(a1), math.sin(a1), 0],
            [-1*math.sin(a1), math.cos(a1), 0],
            [0,0,1]
        ])

        xyz =np.array([x,y,z])
        ## Getting xyz rotated into xz-plane
        new_xz = rotation_matrix.dot(xyz)
        new_xz[0] -= l1

        # (4) Now we are back to standard 2R in plane case
        # Copied code from previous problem
        x, y = new_xz[0], new_xz[1]
        l1, l2= self.l2, self.l3

        # Be careful about 0 division errors
        a3 = math.acos((x**2 + y**2 - l1**2 - l2**2) / (2*l1*l2))
        if x != 0:
            a2 = math.atan(y/x) - math.atan((l2*math.sin(a3)) / (l1 + l2 * math.cos(a3)))
        else: 
            a2 = math.pi/2 - math.atan((l2*math.sin(a3)) / (l1 + l2 * math.cos(a3)))
        
        return (a1, a2, a3)
    


In [49]:
# Creating robot
link_lengths = (10, 40, 30)
testRobot = Robot3R(link_lengths)

print(testRobot.inverse_kinematics((10,20,20)))
print(math.atan(20/10))

(1.1071487177940904, -0.5308079026256183, 2.931471561897582)
1.1071487177940904


**Testing**

*Forward Kinematics*

In [20]:
test1 = testRobot.forward_kinematics((0,0,0))
print(test1, "should be:", (130, 0))

test2 = testRobot.forward_kinematics((math.pi/4, math.pi/4, math.pi/4))
print(test2, "should be", (60*math.sqrt(2)/2 - 30 * math.sqrt(2)/2, 60* math.sqrt(2)/2 + 40 + 30 * math.sqrt(2)/2))

print(testRobot.inverse_kinematics((110,0)))

(130.0, 0.0) should be: (130, 0)
(21.21320343559643, 103.63961030678928) should be (21.213203435596427, 103.63961030678928)
(0.0, 0.6435011087932844, 1.5707963267948966)
